<hr></hr>
Change this parameter to use it on your own file

In [51]:
ifile = 'test_annovar.hg19.tsv'

<hr></hr>

##Parsing ANNOVAR
A new approach @CINECA


* Using [mETL](https://github.com/ceumicrodata/mETL) package
* Configuration file is in [YAML format](http://pyyaml.org/wiki/PyYAMLDocumentation) through python library

### Define fields to be parsed

In [52]:
fields = [
# Chromosome
{'map': 'Chr', 'name': 'chromosome', 'type': 'String'},
# Start
{'map': 'Start', 'name': 'position', 'type': 'Float'},
]

### YAML configuration with python code

Configuration for source and target after decisions on fields

In [53]:
# Create YAML configuration file automatically
import yaml
yfile = 'test.yml'
sqlfile = 'test.db'
table = 'test'
# Init python dict
data = {}
# Input
data['source'] = { 'resource': ifile, 'fields': fields, 'source': 'TSV', 'headerRow': 0, 'skipRows': 1}
# Output
data['target'] = {
    'type': 'Database', 'createTable': 'true', 'replaceTable': 'true', 'truncateTable': 'true',
    'table': table, 'url': 'sqlite:///' + sqlfile,
}

Write the files and check content

In [54]:
# Write the file
with open(yfile, 'w') as outfile:
    outfile.write( yaml.dump(data, default_flow_style=False) )

#!head {yfile} -n 3
out = !! wc -l {yfile}
print "Your configuration inside file '" + yfile + "' reaches " + out.s.split(" ")[0] + " lines"

Your configuration inside file 'test.yml' reaches 19 lines


### Execute the parser

The following use the *metl* package to read the variants annotated and save it into a file database

In [59]:
# Execute via ipython
! metl {yfile} 

### Use the database

Read data in python and manipulate

In [56]:
# init
import sqlite3 as lite
conn = None
# connect
try:
    conn = lite.connect(sqlfile)    
    cur = conn.cursor()     
except lite.Error, e:
    #print "Error %s:" % e.args[0]
    raise BaseException("No sql lite connection", e)
finally:
    print "Connected"

Connected


In [57]:
# Query
query = "SELECT * FROM " + table
cur.execute(query)
print cur.fetchone()


(1, u'chr1', 10385471)


<hr></hr>


* finale notes*

- transformation with python function
- manipulation (filters)
- convert from db to excel for download ;)

In [58]:
# The end, release resources
if conn:
    conn.close()